# Pneumonia detection from X-ray

## downloading tensorflow for the project
##### run the following command in a code sell to download tensorflow



```terminal
!pip install tensorflow
```



### transfer learning is used to make this project 
####  <u>Transfer learning</u> :   Transfer Learning is basically when you train a model on a data and then used that trained model on a diffrent but similer data so that you don't have to train the entire model , which may take a lot of time and GPU power and for a good preformance will also required a large data set, which we know may not be peresent and making our own data set is tedious and expencive task

##### we will be using VGG16 and VGG19

#### you can donload the data set form [here](https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia)

In [2]:
import tensorflow as tf

# Check if GPU is available and set the device
if tf.config.list_physical_devices('GPU'):
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
    device_name = tf.test.gpu_device_name()
    print(f"GPU device name: {device_name}")
else:
    print("No GPU found, using CPU.")

# Example TensorFlow operations using GPU
with tf.device('/GPU:0'):
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    c = tf.matmul(a, b)

print(c)

# Monitor GPU utilization
!nvidia-smi


2024-07-04 12:03:11.469391: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 12:03:11.469512: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 12:03:11.628655: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num GPUs Available:  2
GPU device name: /device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Thu Jul  4 12:03:24 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P0             26W /   70W |   14059MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [12]:
# importing all the necessary libraries
import tensorflow as tf
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential 
import numpy as np
import glob as glob
import matplotlib.pyplot as plt

In [13]:
# re-sizing all the images ,as we are delaing with rgb images so it will have 3 dimentions
IMAGE_SIZE = (224, 224)

train_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'
valid_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/val'

In [19]:
# using vgg16 model , and we will be using imagenet weights, as they were the one who win the competion and so ther weights 
# are of the trained modle so we won't have to train the whole model , this is transfer learnig 


from tensorflow.keras.applications import DenseNet121

DenseNet = DenseNet121(weights='imagenet', include_top=False, input_shape= (224, 224, 3))

# [3] is for the dimention of the rgb imgae

# include_top = False means we don't want the exact model as we are going to use a diffrent data set so , we will train 
# the the last layer and the output layer for out dataset so that the model predicts for this particular data set
# in the resNet50 starting layers of the cnn identify the basic pattrens that are persent in all the images like lines, edges etc, 
# so we don't need to change the staring layers for that we have keept them as same and removed the top part that we will be 
# training for out data set

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [20]:
# we are loading a pretrained model so we don't want to change the weights that we have laoded in the avobe code cell 
for layer in DenseNet.layers:
        layer.trainable = False    # so that we don't train the layers

In [21]:
from glob import glob
folders = glob("/kaggle/input/chest-xray-pneumonia/chest_xray/train/*")

# this will tell us which folders are present in Data_set/train , * repersents all , 

In [22]:
folders

['/kaggle/input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA',
 '/kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL']

In [23]:
from tensorflow.keras.layers import Dropout, GlobalAveragePooling2D

x = GlobalAveragePooling2D()(DenseNet.output)

# Add fully connected layers
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)
# here flatten is bsically reducing the dimentionality of the layers , making it in 1D
# vgg.output is the output layer and we are flattening the output layre

In [24]:
prediction = Dense(len(folders), activation= "softmax")(x)
# x is our output layer and we are usign Dense function to five it no. of neurons that there are in the train file
# for this case there are only two , pneumonia and not pneumonia 
# we are using activatin function softmax

model = Model(inputs = DenseNet.input, outputs = prediction)
# we are giving the model the inputs that are pretrained form vgg and outputs according to out data set

In [25]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,408 │ zero_padding2d[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_1    │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 56, 56,    │          0 │ zero_padding2d_1… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, 56, 56,    │          0 │ conv2_block1_0_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, 56, 56,    │          0 │ pool1[0][0],      │
│ (Concatenate)       │ 96)               │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, 56, 56,    │        384 │ conv2_block1_con… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_relu │ (None, 56, 56,    │          0 │ conv2_block2_0_b… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_conv │ (None, 56, 56,    │     12,288 │ conv2_block2_0_r

 Total params: 7,694,146 (29.35 MB)

 Trainable params: 656,642 (2.50 MB)

 Non-trainable params: 7,037,504 (26.85 MB)

In [26]:
# now we will compile the model , i.e. giving it all the things it will need to train well and make good predictions 
from tensorflow.keras.optimizers import Adam
model.compile(loss = 'categorical_crossentropy', 
             optimizer = Adam(0.0001),
             metrics = ["accuracy"])

In [27]:
# using image generator to import the images form the data set


train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2, 
                                  horizontal_flip=True)

# here we are configuring the ImageDataGenerator with the parameters like rescale and all so that our model performs well
# it is adviced that if all the data is in same range then its better for the model 
### the basic reason we are doing this is because we want to increase out tring data somehow so that our model doesn't overfit
### this data augmentation is helping us with generarating more data for the training 

# rescale = 1./255 is rescaling the image's pixel value that ranges form 0 to 255 by deviding by 255 we can make it in range of 
# [0,1], this is a convention for better training 

# shear_range=0.2 shearing means keeping one part of the image fixed and move the other part in one direction keeping the 
# image stationary, this can introduce diversity into the training data

# horizontal_flip is used to mirror image this helps in making the model invarient to horizontal flips


test_datagen = ImageDataGenerator(rescale = 1./255)

In [28]:
# training data
# make sure to provide the same target size as initialized before 
training_set = train_datagen.flow_from_directory("/kaggle/input/chest-xray-pneumonia/chest_xray/train",
                                                target_size=(224,224),
                                                batch_size=32,
                                                class_mode= "categorical")

# at once 32 images will be given for training and we have to categorize the images 
# target size is (244, 244) this is done to ensure that the size of all the images is same when being fed to the model


Found 5216 images belonging to 2 classes.


In [29]:
# doing the same for test data
test_set = test_datagen.flow_from_directory("/kaggle/input/chest-xray-pneumonia/chest_xray/test",
                                                target_size=(224,224),
                                                batch_size=32,
                                                class_mode= "categorical")

Found 624 images belonging to 2 classes.


In [30]:
print(len(test_set))
print(next(iter(test_set))[0].shape, next(iter(test_set))[1].shape)

20
(32, 224, 224, 3) (32, 2)


In [31]:
# now we train our model 
with tf.device('/GPU:0'):
    xray = model.fit(
        training_set,
        validation_data=test_set,
        epochs=25,
        steps_per_epoch=len(training_set),
        validation_steps=(len(test_set)-1)
    )

Epoch 1/25


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  1/163 ━━━━━━━━━━━━━━━━━━━━ 1:55:45 43s/step - accuracy: 0.5312 - loss: 1.0790

I0000 00:00:1720095072.747130     105 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1720095072.821833     105 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 666ms/step - accuracy: 0.7356 - loss: 0.6261

...

163/163 ━━━━━━━━━━━━━━━━━━━━ 167s 768ms/step - accuracy: 0.7360 - loss: 0.6251 - val_accuracy: 0.7780 - val_loss: 0.4444
Epoch 2/25


/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


163/163 ━━━━━━━━━━━━━━━━━━━━ 15s 94ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8125 - val_loss: 0.4235
Epoch 3/25


W0000 00:00:1720095212.516416     105 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


163/163 ━━━━━━━━━━━━━━━━━━━━ 95s 559ms/step - accuracy: 0.8949 - loss: 0.2521 - val_accuracy: 0.7763 - val_loss: 0.4891
Epoch 4/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 591us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6875 - val_loss: 0.6189
Epoch 5/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 95s 562ms/step - accuracy: 0.9048 - loss: 0.2279 - val_accuracy: 0.8322 - val_loss: 0.3705
Epoch 6/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 534us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9375 - val_loss: 0.1404
Epoch 7/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 96s 566ms/step - accuracy: 0.9195 - loss: 0.1925 - val_accuracy: 0.8487 - val_loss: 0.3472
Epoch 8/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 615us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8125 - val_loss: 0.3296
Epoch 9/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 96s 561ms/step - accuracy: 0.9337 - loss: 0.1764 - val_accuracy: 0.8717 - val_loss: 0.3094
Epoch 10/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 870us/step - accuracy: 0.0000

In [33]:
# from keras.models import load_model
model.save("/kaggle/working/DenseNet_pneumonia_epoch_25.h5")

In [34]:
from keras.models import load_model
loaded_model = load_model("DenseNet_pneumonia_epoch_25.h5")

In [59]:
# image has to be converted into numbers --> array to before to be processed

img = image.load_img( "/kaggle/input/chest-xray-pneumonia/chest_xray/val/NORMAL/NORMAL2-IM-1442-0001.jpeg", target_size = (224,224))
x = image.img_to_array(img)
x = np.expand_dims(img, axis = 0)
img_data = preprocess_input(x)
ans = loaded_model.predict(img_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


In [60]:
ans

array([[1.9539859e-17, 1.0000000e+00]], dtype=float32)

In [61]:
predicted_class = np.argmax(ans)
predicted_class

1

In [62]:
if predicted_class == 0:
    print("you dont have Pneumonia")
else:
    print("Sorry, you have Pneumonia")

Sorry, you have Pneumonia


In [64]:
predictions = model.predict(test_set)
print(predictions[:20])  # Print first 10 predictions

 2/20 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step 

W0000 00:00:1720097354.774100     106 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


20/20 ━━━━━━━━━━━━━━━━━━━━ 24s 751ms/step
[[3.3703570e-03 9.9662966e-01]
 [5.8846432e-03 9.9411535e-01]
 [1.0012654e-02 9.8998731e-01]
 [3.0337687e-04 9.9969655e-01]
 [6.5470308e-01 3.4529701e-01]
 [3.7045893e-04 9.9962950e-01]
 [6.5376204e-03 9.9346244e-01]
 [4.7366734e-04 9.9952632e-01]
 [3.7426266e-03 9.9625742e-01]
 [2.8018304e-03 9.9719822e-01]]


W0000 00:00:1720097369.034095     106 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [66]:
print(predictions[:20])

[[3.3703570e-03 9.9662966e-01]
 [5.8846432e-03 9.9411535e-01]
 [1.0012654e-02 9.8998731e-01]
 [3.0337687e-04 9.9969655e-01]
 [6.5470308e-01 3.4529701e-01]
 [3.7045893e-04 9.9962950e-01]
 [6.5376204e-03 9.9346244e-01]
 [4.7366734e-04 9.9952632e-01]
 [3.7426266e-03 9.9625742e-01]
 [2.8018304e-03 9.9719822e-01]
 [4.1536218e-03 9.9584639e-01]
 [5.7681110e-03 9.9423188e-01]
 [1.3303217e-04 9.9986696e-01]
 [2.4687756e-02 9.7531223e-01]
 [1.2378132e-03 9.9876225e-01]
 [4.4693876e-02 9.5530611e-01]
 [4.1772588e-04 9.9958223e-01]
 [8.8975871e-01 1.1024124e-01]
 [2.1525232e-04 9.9978471e-01]
 [9.7797567e-01 2.2024367e-02]]
